# Traitement des données de la commande publique

** Prétraitement des données de la commande publique pour le hackathon Etalab du Forum pour la paix **

In [ ]:
from bs4 import BeautifulSoup
import urllib
import re
import requests
from urllib.request import urlopen
import codecs
import xml.etree.ElementTree as etree
import pandas as pd
import bs4
import json
import os
import numpy as np
import unicodedata

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

## Liste des services de l'annuaire

Boucle sur le répertoire **repo** défini ci-dessus pour extraire tous les liens xml. 
/!\ Il faudra traiter séparément le cas des json (actuellement, 4 fichiers tous xml)

In [ ]:
racine = "https://lannuaire.service-public.fr/"
branches = ["gouvernement", "institutions-juridictions", "autorites-independantes", "ambassades", "institutions-europeennes"]
branches = [racine + branche for branche in branches]
branches_sauvegarde = []

In [ ]:
while len(branches) >0 : 
    print('Nouveau tour')
    branches_new = []
    for i, url in enumerate(branches) :
        print(i, url)
        branches_sauvegarde.append(url)
        
        html_page = urllib.request.urlopen(url)
        soup = BeautifulSoup(html_page, 'lxml')

        # Organisations
        for l0 in soup.find_all('li', itemprop = "Organization") :
            for l1 in l0.find_all('a') : 
                lien = l1['href']
                if lien not in branches_sauvegarde and lien not in branches_new: # Pour éviter les boucles à répétition
                    branches_new.append(lien)
    branches = branches_new              

In [ ]:
print(len(branches_sauvegarde))

## Chargement des pages

In [ ]:
with open('liens_bruts.txt', 'w') as f:
    for item in branches_sauvegarde:
        f.write("%s\n" % item)

In [ ]:
if not os.path.exists("Pages"):
    os.makedirs("Pages")

In [ ]:
import urllib.request
import re

for url in branches_sauvegarde :
    url_light = re.sub('^https://lannuaire.service-public.fr/', '', url)
    url_light = url_light.replace('/', '_')
    
    nom_sauvegarde = 'Pages/' + url_light + '.html'
    print(nom_sauvegarde)
    if nom_sauvegarde == ".html" :
        continue
    urllib.request.urlretrieve(url, nom_sauvegarde)

In [ ]:
from os import listdir
from os.path import isfile, join
pages_chargees = [f for f in listdir('Pages/') if isfile(join('Pages/', f))]
pages_chargees[0:10]

In [ ]:
url_light = [re.sub('^https://lannuaire.service-public.fr/', '', f) for f in branches_sauvegarde]
url_light = [f.replace('/', '_') + ".html" for f in url_light]

In [ ]:
non_telechargees = list(set(pages_chargees) - set(url_light))
non_telechargees

## Extraction des informations

In [ ]:
annuaire = []

In [ ]:
for page in pages_chargees : 
    nom_complet = "Pages/" + page
    #print(nom_complet)
    f = codecs.open(nom_complet, 'r', 'utf-8')
    document = BeautifulSoup(f.read(), "lxml")

    #Hiérarchie
    hierarchie = document.findAll('div', class_ = "breadcrumb")[0].text
    hierarchie = hierarchie.split('>')[1:-1]
    hierarchie = [s.replace('\xa0', '') for s in hierarchie]
    hierarchie = " > ".join(hierarchie)

    # Nom du service
    service = document.findAll('h1', id = "contentTitle")[0].text

    #Boucle sur les personnes
    personnes = document.findAll('p', itemprop = "jobTitle")
    for personne in personnes :

        try : 
            titre = personne.text
        except : 
            titre = ''
        try : 
            nom = personne.next_sibling.text
        except : 
            nom = ''
        entree = {"page" : nom_complet, "nom_service" : service, "hierarchie" : hierarchie, "titre" : titre, "personne" : nom}
        annuaire.append(entree)

In [ ]:
annuaire = pd.DataFrame(annuaire)
annuaire = annuaire.replace("\\n",' ', regex=True) 
annuaire.to_csv('annuaire.csv', sep=';', encoding='utf-8', index=False)

## Post-traitement pour nettoyer, uniformiser

In [ ]:
annuaire = pd.read_csv('annuaire.csv', header=0, sep = ';')
#annuaire = annuaire.drop(['page', 'hierarchie', 'nom_service'], axis=1)

# Si identité absente
annuaire['personne'] = annuaire['personne'].fillna('Non communiqué')
annuaire.head()

In [ ]:
# Ajout d'un ordre d'apparition sur la page
annuaire['rang'] = annuaire.groupby('page').cumcount()
annuaire['profondeur'] = annuaire.apply(lambda row : row['hierarchie'].count('>'), axis = 1)
annuaire.head()

In [ ]:
def retirerCaracteresSpeciaux(chaine) :
    chaine = unicodedata.normalize('NFD', chaine).encode('ascii', 'ignore').decode('utf-8')
    return(chaine)

In [ ]:
def separerTitre (chaine) :
    regex_Titre = '^(.*), (.*)$'
    if re.search(regex_Titre, chaine) : 
        return(re.search(regex_Titre, chaine).groups())
    else : 
        return(['',''])

In [ ]:
def separerPrenomNomTitre (chaine) :
    regex_prenomNomTitre = '^([A-Za-z \-\']*?) ([A-Z \-\']{2,})(|, *)($|[,A-Za-z -\']*)'
    if re.search(regex_prenomNomTitre, chaine) : 
        return(re.search(regex_prenomNomTitre, chaine).groups())
    else : 
        return(['', '','', ''])

In [ ]:
annuaire['personne'] = annuaire.apply(lambda row: retirerCaracteresSpeciaux(row['personne']), axis=1)
annuaire['titre'] = annuaire.apply(lambda row: retirerCaracteresSpeciaux(row['titre']), axis=1)

In [ ]:
annuaire['prenom'] = annuaire.apply(lambda row: separerPrenomNomTitre(row['personne'])[0], axis=1)
annuaire['nom'] = annuaire.apply(lambda row: separerPrenomNomTitre(row['personne'])[1], axis=1)
annuaire['corps'] = annuaire.apply(lambda row: separerPrenomNomTitre(row['personne'])[3], axis=1)
annuaire['corps'] = annuaire.apply(lambda row: row['corps'].replace(', ', ''), axis=1)

In [ ]:
annuaire['prenom'] = annuaire['prenom'].str.upper()
annuaire.head()

In [ ]:
annuaire.shape

### Chargement et prétraitement de la base prénom

In [ ]:
# Chargement de la base prenoms
prenoms = pd.read_csv('nat2017.txt', sep = '\t')
prenoms['sexe'] = prenoms['sexe'].map({2 : 'female', 1: 'male'})
prenoms = prenoms.groupby(['sexe', 'preusuel'])['nombre'].sum().reset_index()
prenoms['preusuel'] = prenoms.apply(lambda row: retirerCaracteresSpeciaux(row['preusuel']), axis=1)
prenoms = prenoms.groupby(['sexe', 'preusuel'])['nombre'].sum().reset_index() # On somme à nouveau car variantes dues aux accents

prenoms = prenoms.pivot(index = 'preusuel', columns = 'sexe', values = 'nombre').reset_index()

prenoms = prenoms.fillna(0)
prenoms['total'] = prenoms['female'] + prenoms['male']
prenoms['propMale'] = prenoms['male']/prenoms['total']

prenoms.head()

In [ ]:
prenoms.to_csv('prenoms_light.csv', sep=';', encoding='utf-8', index=False)
print(prenoms.shape)

### Jointure annuaire / prénoms

In [ ]:
annuaire_genre = annuaire.merge(prenoms[['preusuel', 'propMale']], left_on='prenom', right_on='preusuel', 
                      left_index = True, how = 'left', indicator = True)
annuaire_genre = annuaire_genre.reset_index()
annuaire_genre = annuaire_genre.drop(['index'], axis=1)

annuaire_genre['genre'] = np.NaN
annuaire_genre.loc[annuaire_genre['propMale']>0.95, 'genre'] = 'M'
annuaire_genre.loc[annuaire_genre['propMale']<0.02, 'genre'] = 'F'
annuaire_genre.head()

In [ ]:
titreF = ['directrice', 'cheffe', 'conseillere', 'presidente', 'generale', 'deleguee',
 'chargee', 'agente', 'adjointe', 'ambassadrice', 'consule', 'administratrice', 'elue']
titreM = ['ambassadeur', 'president', 'chef', 'directeur', 'controleur', 'conseiller', 'inspecteur', 
 'adjoint', 'controleur', 'delegue', 'charge', 'coordonnateur', 'general','agent', 'elu', 'commandant', 'lieutenant']

prenomsF = ['MARITSA', 'CERA', 'URWANA', 'EVGENIA', 'YASMINE-EVA', 'EDWIGE', 'HELEN']
prenomsH = ['ETIENNE-MARTIN', 'GARIN', 'THIERRY-OLIVIER', 'VALERY']

In [ ]:
for i, row in annuaire_genre.iterrows():
    
    # Si mot féminin présent dans le titre
    if pd.isnull(row["genre"]) and row['personne'] != 'Non communique':
        mots = row["titre"].replace(',',' ').replace('-', ' ').split()
        mots = [m.lower() for m in mots]
        
        #print(mots)
        if any(titre_termes in mots for titre_termes in titreF) or row['prenom'] in prenomsF :
            #print('femme  ', row['titre'])
            annuaire_genre.loc[i,'genre'] = 'F'
        elif any(titre_termes in mots for titre_termes in titreM) or row['prenom'] in prenomsH :
            #print('homme  ', row['titre'])
            annuaire_genre.loc[i,'genre'] = 'M'
        # On tranche en affectant le genre le plus fréquent du nom
        else : 
            if annuaire_genre.loc[i,'propMale']> 0.5 : 
                annuaire_genre.loc[i,'genre'] = 'M'
            else :
                annuaire_genre.loc[i,'genre'] = 'F'
        

In [ ]:
restant = annuaire_genre[(pd.isnull(annuaire_genre['genre'])) & (annuaire_genre['personne'] != "Non communique")]
restant

In [ ]:
annuaire_genre.to_csv('annuaire_genre.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
annuaire_genre[annuaire_genre['prenom'] == 'CAMILLE']

## Premières analyses

In [ ]:
def traitementHierarchie (chaine) :
    chaine = chaine.split('>')
    chaine = [e.lstrip().rstrip() for e in chaine]
    chaine = [e for e in chaine if e != '']
    chaine = list(set(chaine))

    return([chaine, len(chaine)])

In [ ]:
annuaire_genre['profondeur'] = annuaire_genre.apply(lambda row : traitementHierarchie(row['hierarchie'])[1], axis = 1)

In [ ]:
annuaire_genre.head()

In [ ]:
#annuaire_genre.loc[(annuaire_genre['rang']==0)], ['genre', 'profondeur']].value_counts()
ratio = annuaire_genre.loc[annuaire_genre['rang']<1].groupby(['genre', 'profondeur']).size().reset_index()
ratio.columns = ['genre', 'profondeur', 'nombre']
ratio = ratio.pivot(index = 'profondeur', columns = 'genre', values = 'nombre').reset_index()

ratio = ratio.fillna(0)
ratio['propMale'] = ratio['M']/(ratio['F'] + ratio['M'])
ratio

In [ ]:
1862/(1862+3669)